In [1]:
# # Install Pytorch & other libraries, make sure to match your GPU driver version
# %pip install "torch==2.5.1" tensorboard "setuptools<71.0.0"  --index-url https://download.pytorch.org/whl/cu121

# # Install flash-attn
# %pip install flash-attn

# # Install Hugging Face libraries
# %pip install  --upgrade \
#   "transformers==4.48.1" \
#   "datasets==3.1.0" \
#   "accelerate==1.3.0" \
#   "hf-transfer==0.1.9" \
#   "deepspeed==0.15.4" \
#   "trl==0.14.0"

# # install vLLM
# %pip install "vllm==0.7.0"

# # ## IMPORTANT: If you want to run the notebook and the interactive cells you also need to install the following libraries:
# # # But first read it the blog post and then decide as they might conflict with the libraries for distributed training.
# # # %pip install "peft==0.14.0" "bitsandbytes==0.45.0"

In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [3]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import LoraConfig
from trl import GRPOConfig, GRPOTrainer
import numpy as np

[2025-04-18 12:28:59,064] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3

INFO 04-18 12:29:00 __init__.py:183] Automatically detected platform cuda.


In [ ]:
# tensorboard --port 6007 --logdir /root/tf-logs/
# ps -ef | grep tensorboard | awk '{print $2}' | xargs kill -9 
# tensorboard --logdir=/root/autodl-tmp/outputs/Qwen-0.5B-GRPO/runs/Apr18_12-29-07_autodl-container-ef014e83a6-5e7d45d3 --port=6007
# tensorboard --logdir "C:\Users\Yuan\Desktop\YUAN\學業\北京清華\碩一下\LLM\Qwen-0.5B-GRPO\GRPO limited COT ver.3 Anum3 Gnum6"

# autodl-tmp/outputs/Qwen-0.5B-GRPO/runs/Apr16_16-34-38_autodl-container-ef014e83a6-5e7d45d3

In [5]:
# Load and prep dataset

SYSTEM_PROMPT = """
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip().replace(",", "").replace("$", "")

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': 'What is the largest single-digit prime number?'},
            {'role': 'assistant', 'content': XML_COT_FORMAT.format(
               reasoning="9 is divisble by 3 and 8 is divisible by 2, but 7 is prime.",
               answer="7"
            )},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# dataset = load_dataset(
#     'parquet',
#     data_files={
#         'train': 'gsm8k/train/train-00000-of-00001.parquet',
#         'test': 'gsm8k/test/test-00000-of-00001.parquet'
#     }
# )

# Reward functions

def accuracy_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [1.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.25 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses] 
    return [0.25 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r, flags=re.DOTALL) for r in responses] 
    return [0.25 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

def length_cosine_reward_func(completions, answer, prompts, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_answers = [extract_xml_answer(r) for r in responses]

    rewards = []
    for i, r in enumerate(responses):  # 對於每一個模型輸出的回答 response（r）
        reasoning_text = r.split("<answer>")[0]  # 只取出 `<answer>` 之前的部分（即 reasoning 推理內容）
        
        # 使用 tokenizer 將 reasoning_text 分詞成 tokens，並取其長度作為 reasoning 的 token 長度
        reasoning_len = len(tokenizer(reasoning_text, return_tensors="pt")["input_ids"][0])
        
        is_correct = extracted_answers[i] == answer[i]  # 判斷模型的最終回答是否與正確答案相符（是否正確）

        # 設定最大長度為 16000，將推理長度 reasoning_len 映射到 [0, π] 的範圍（正規化）
        max_length = 786 
        x = np.clip(reasoning_len / max_length, 0, 1) * np.pi/2  # 確保比例不超過 1，避免過長失控

        if is_correct:
            # 如果答對了，就使用「遞減型餘弦」，鼓勵短的答案（reward 隨長度增加而減少）
            reward = 5 * np.cos(x)  # 最多給 +5，越長越接近 0
        else:
            # 如果答錯了，就使用「反向餘弦」，鼓勵多一點思考（reward 隨長度增加而上升）
            # reward = -5.0 * np.cos(x)  # 最多扣 -10，越長越接近 0（或甚至轉為正）
            reward = 0
        rewards.append(float(reward))  # 把這一筆的 reward 加入總 reward 清單
    return rewards  # 回傳所有 sample 的 reward 分數

In [6]:
# # import torch
# from importlib.metadata import version

# # 检查 PyTorch 和 CUDA 版本
# print(f"PyTorch 版本: {torch.__version__}")
# print(f"CUDA 可用: {torch.cuda.is_available()}")
# print(f"CUDA 版本: {torch.version.cuda}")

# # 检查其他库版本
# libraries = [
#     "transformers", "datasets", "accelerate",
#     "deepspeed", "trl", "vllm", "peft", 
#     "bitsandbytes", "flash-attn"
# ]

# for lib in libraries:
#     try:
#         print(f"{lib} 版本: {version(lib)}")
#     except:
#         print(f"{lib} 未安装")

In [7]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class CustomSaveCallback(TrainerCallback):
    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step >= 1500 and state.global_step % 100 == 0:
            control.should_save = True
        else:
            control.should_save = False
        return control


In [ ]:
#model_name = "meta-llama/Llama-3.2-1B-Instruct"

model_name = "models/Qwen2.5-0.5B-Instruct"

if "Llama" in model_name:
    output_dir = "outputs/Llama-1B-GRPO"
    run_name = "Llama-1B-GRPO-gsm8k"
else:
    output_dir="outputs/Qwen-0.5B-GRPO"
    run_name="Qwen-0.5B-GRPO-gsm8k"
    
training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=2e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=3,
    num_generations=6,
    max_prompt_length=256,
    max_completion_length=786,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    report_to="tensorboard",
    log_on_each_node=False,
)
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=64,
#     target_modules=["q_proj", "k_proj", "v_proj", "ao_proj", "up_proj", "down_proj", "gate_proj"],
#     task_type="CAUSAL_LM",
#     lora_dropout=0.05,
# )
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map=None
).to("cuda")
        
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        length_cosine_reward_func,
        accuracy_func],
    args=training_args,
    train_dataset=dataset,
    callbacks=[CustomSaveCallback()],
    #peft_config=peft_config
)
trainer.train()

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
The total number of points Ahmed and Emily are worth in the class is 9*91+92=1050. Emily has already made 90 on the final assignment, and has 9*4=36 total points. Ahmed needs 1050-36=1014. The minimum grade Ahmed needs to get is 1014/91=~11.12

Therefore, the answer is 11.
</reasoning>
<answer>
11
</answer> 
Extracted:
11
-------------------- Question:
The gauge on a water tank shows that the tank is 1/3 full of water. To fill the tank, 16 gallons of water are added. How many gallons of water does the tank hold when full? 
Answer:
24 
Response:
Since the tank is 1/3 

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


-------------------- Question:
Fiona completed 36 math questions in an hour. Shirley was able to complete twice as many math questions within that same time, and Kiana completed half of the sum of Fiona and Shirley's math questions. If they each did the same number of questions the following hour, how many math questions did all three girls complete in 2 hours? 
Answer:
324 
Response:
Fiona's questions: 36
Shirley's questions: 36 * 2 = 72
Fiona + Shirley = 36 + 72 = 108
Kiana's questions: 108 / 2 = 54
Fiona's / Shirley's questions = 108 / 72 = 1.5
Therefore the 3 total questions: 1.5 * 2 = 3.

Therefore, the answer is 3. 
Extracted:
Fiona's questions: 36
Shirley's questions: 36 * 2 = 72
Fiona + Shirley = 36 + 72 = 108
Kiana's questions: 108 / 2 = 54
Fiona's / Shirley's questions = 108 / 72 = 1.5
Therefore the 3 total questions: 1.5 * 2 = 3.

Therefore, the answer is 3.
-------------------- Question:
In a graveyard, there are 20 skeletons.  Half of these skeletons are adult women, and t